In [146]:
import h5py
import pandas as pd
import numpy as np

In [147]:
filename = 'processed-pythia82-lhc13-all-pt1-50k-r1_h022_e0175_t220_nonu_withPars_truth_9'

In [148]:
f = h5py.File('raw/' + filename + '.z', 'r')
treeArray = f['t_allpar_new'][()]
f.close()
print(treeArray.dtype.names)

('index', 'j_ptfrac', 'j_pt', 'j_eta', 'j_mass', 'j_tau1_b1', 'j_tau2_b1', 'j_tau3_b1', 'j_tau1_b2', 'j_tau2_b2', 'j_tau3_b2', 'j_tau32_b1', 'j_tau32_b2', 'j_zlogz', 'j_c1_b0', 'j_c1_b1', 'j_c1_b2', 'j_c2_b1', 'j_c2_b2', 'j_d2_b1', 'j_d2_b2', 'j_d2_a1_b1', 'j_d2_a1_b2', 'j_m2_b1', 'j_m2_b2', 'j_n2_b1', 'j_n2_b2', 'j_tau1_b1_mmdt', 'j_tau2_b1_mmdt', 'j_tau3_b1_mmdt', 'j_tau1_b2_mmdt', 'j_tau2_b2_mmdt', 'j_tau3_b2_mmdt', 'j_tau32_b1_mmdt', 'j_tau32_b2_mmdt', 'j_c1_b0_mmdt', 'j_c1_b1_mmdt', 'j_c1_b2_mmdt', 'j_c2_b1_mmdt', 'j_c2_b2_mmdt', 'j_d2_b1_mmdt', 'j_d2_b2_mmdt', 'j_d2_a1_b1_mmdt', 'j_d2_a1_b2_mmdt', 'j_m2_b1_mmdt', 'j_m2_b2_mmdt', 'j_n2_b1_mmdt', 'j_n2_b2_mmdt', 'j_mass_trim', 'j_mass_mmdt', 'j_mass_prun', 'j_mass_sdb2', 'j_mass_sdm1', 'j_multiplicity', 'j1_px', 'j1_py', 'j1_pz', 'j1_e', 'j1_pdgid', 'j1_erel', 'j1_pt', 'j1_ptrel', 'j1_eta', 'j1_etarel', 'j1_etarot', 'j1_phi', 'j1_phirel', 'j1_phirot', 'j1_deltaR', 'j1_costheta', 'j1_costhetarel', 'j1_e1mcosthetarel', 'j_g', 'j_q', 

In [149]:
features = ["j1_e","j1_px","j1_py","j1_pz"]
j_index = ["j_index"]
labels = ['j_q', 'j_g', 'j_w', 'j_z', 'j_t']

In [150]:
df = pd.DataFrame(treeArray,columns=list(set(features+j_index)))
df = df.drop_duplicates()
df = df[features+j_index]

In [151]:
df2 = (df.pivot_table(index = "j_index", columns = df.groupby("j_index").cumcount().add(1), aggfunc = "first", fill_value = 0).sort_index(axis = 1, level = 1))
df2 = (df2.set_axis([f'{x}{y}' for x,y in df2.columns], axis=1).reset_index())
df2 = df2.iloc[:, 1:801]
df2.reset_index(drop = True, inplace = True)

In [152]:
if(len(df2.columns) < 800):
    i = int(len(df2.columns) / 4)
    length = df2.shape[0]
    while i <= 200:
        df2["j1_e" + str(i)] = np.zeros((length))
        df2["j1_px" + str(i)] = np.zeros((length))
        df2["j1_py" + str(i)] = np.zeros((length))
        df2["j1_pz" + str(i)] = np.zeros((length))
        i = i + 1

In [153]:
df2

,j1_e1,j1_px1,j1_py1,j1_pz1,j1_e2,j1_px2,j1_py2,j1_pz2,j1_e3,j1_px3,...,j1_py198,j1_pz198,j1_e199,j1_px199,j1_py199,j1_pz199,j1_e200,j1_px200,j1_py200,j1_pz200
0,750.853638,263.664520,-182.444260,-678.952332,585.144897,205.583252,-143.016251,-528.844421,220.193924,76.962601,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,432.886475,-8.857548,354.048767,-248.921112,204.682266,-3.753098,166.691681,-118.720779,160.568924,-2.296667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,355.651337,255.912674,11.529702,-246.705582,81.910553,59.158012,2.915969,-56.578671,79.103516,57.111702,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,533.614868,507.421356,48.375240,157.886826,50.898926,49.697281,3.225070,10.468827,36.955311,35.339100,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,68.106377,0.791573,-63.289627,-25.145077,53.902973,-2.909187,-52.814068,-10.379848,50.724018,-2.523490,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98763,162.574478,66.458244,146.757843,-21.815189,149.549133,61.525631,134.908997,-19.470064,102.112190,41.342888,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98764,99.811035,55.350876,81.288795,-17.048637,44.799080,24.960217,36.451290,-7.431595,42.886120,23.469486,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98765,325.679840,-241.943481,-103.064125,192.115845,144.435120,-107.709503,-46.266567,84.377434,101.453598,-75.606232,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98766,137.944458,-90.294876,43.407539,-94.822327,105.007225,-69.811890,32.066605,-71.585968,91.719185,-60.037533,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [154]:
j_px = treeArray["j_pt"]
j_py = np.zeros((treeArray["index"].size))
j_pz = treeArray["j_pt"] * np.sinh(treeArray["j_eta"])
j_E = np.sqrt(np.square(treeArray["j_mass"]) + np.square(treeArray["j_pt"]) + np.square(j_pz))
features_np = np.stack((j_E, j_px, j_py, j_pz, treeArray["j_q"], treeArray["j_g"], treeArray["j_w"], treeArray["j_z"], treeArray["j_t"], treeArray["j_index"]))

In [155]:
features_np = features_np.T

In [156]:
features_df = pd.DataFrame(features_np, columns = ["j_E", "j_px", "j_py", "j_pz", "j_q", "j_g", "j_w", "j_z", "j_t", "j_index"])
features_df = features_df.drop_duplicates()
features_df.sort_values(by=["j_index"], inplace=True)
features_df = features_df.iloc[:, :-1]
features_df.reset_index(drop = True, inplace = True)

In [157]:
features_df

,j_E,j_px,j_py,j_pz,j_q,j_g,j_w,j_z,j_t
0,2449.131348,1043.848389,0.0,-2214.812012,0.0,1.0,0.0,0.0,0.0
1,1430.415771,1169.579834,0.0,-820.326904,0.0,1.0,0.0,0.0,0.0
2,1180.275757,846.473694,0.0,-820.773499,0.0,1.0,0.0,0.0,0.0
3,1276.189209,1214.276001,0.0,374.392609,0.0,1.0,0.0,0.0,0.0
4,1129.609985,1081.496826,0.0,-299.945129,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
98763,974.835327,959.646423,0.0,-94.632500,0.0,0.0,0.0,0.0,1.0
98764,994.302185,975.270325,0.0,-34.542484,0.0,0.0,0.0,0.0,1.0
98765,1139.156982,947.807678,0.0,624.912598,0.0,0.0,0.0,0.0,1.0
98766,1332.987915,955.398376,0.0,-926.830383,0.0,0.0,0.0,0.0,1.0


In [158]:
df_final = pd.concat([df2, features_df], axis=1)
df_final.insert(loc=804, column="ttv", value=0)
df_final["j_q"] = df_final["j_q"].astype(int)
df_final["j_g"] = df_final["j_g"].astype(int)
df_final["j_w"] = df_final["j_w"].astype(int)
df_final["j_z"] = df_final["j_z"].astype(int)
df_final["j_t"] = df_final["j_t"].astype(int)

In [159]:
columns = []
for i in range(200):
    columns.append("E_" + str(i))
    columns.append("PX_" + str(i))
    columns.append("PY_" + str(i))
    columns.append("PZ_" + str(i))
columns.append("truthE")
columns.append("truthPX")
columns.append("truthPY")
columns.append("truthPZ")
columns.append("ttv")
columns.append("is_signal_q")
columns.append("is_signal_g")
columns.append("is_signal_w")
columns.append("is_signal_z")
columns.append("is_signal_t")

In [160]:
df_final.columns = columns

In [161]:
df_final

,E_0,PX_0,PY_0,PZ_0,E_1,PX_1,PY_1,PZ_1,E_2,PX_2,...,truthE,truthPX,truthPY,truthPZ,ttv,is_signal_q,is_signal_g,is_signal_w,is_signal_z,is_signal_t
0,750.853638,263.664520,-182.444260,-678.952332,585.144897,205.583252,-143.016251,-528.844421,220.193924,76.962601,...,2449.131348,1043.848389,0.0,-2214.812012,0,0,1,0,0,0
1,432.886475,-8.857548,354.048767,-248.921112,204.682266,-3.753098,166.691681,-118.720779,160.568924,-2.296667,...,1430.415771,1169.579834,0.0,-820.326904,0,0,1,0,0,0
2,355.651337,255.912674,11.529702,-246.705582,81.910553,59.158012,2.915969,-56.578671,79.103516,57.111702,...,1180.275757,846.473694,0.0,-820.773499,0,0,1,0,0,0
3,533.614868,507.421356,48.375240,157.886826,50.898926,49.697281,3.225070,10.468827,36.955311,35.339100,...,1276.189209,1214.276001,0.0,374.392609,0,0,1,0,0,0
4,68.106377,0.791573,-63.289627,-25.145077,53.902973,-2.909187,-52.814068,-10.379848,50.724018,-2.523490,...,1129.609985,1081.496826,0.0,-299.945129,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98763,162.574478,66.458244,146.757843,-21.815189,149.549133,61.525631,134.908997,-19.470064,102.112190,41.342888,...,974.835327,959.646423,0.0,-94.632500,0,0,0,0,0,1
98764,99.811035,55.350876,81.288795,-17.048637,44.799080,24.960217,36.451290,-7.431595,42.886120,23.469486,...,994.302185,975.270325,0.0,-34.542484,0,0,0,0,0,1
98765,325.679840,-241.943481,-103.064125,192.115845,144.435120,-107.709503,-46.266567,84.377434,101.453598,-75.606232,...,1139.156982,947.807678,0.0,624.912598,0,0,0,0,0,1
98766,137.944458,-90.294876,43.407539,-94.822327,105.007225,-69.811890,32.066605,-71.585968,91.719185,-60.037533,...,1332.987915,955.398376,0.0,-926.830383,0,0,0,0,0,1


In [162]:
df_final.to_hdf('pandas/' + filename + '.h5', key='table', mode='w', format='table')